In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sklearn
plt.ion()

import sklearn.decomposition
import sklearn.svm
import sklearn.neighbors

In [ ]:
## ici on vous donne une fonction très basique qui réalise un train/validation/split 
## il n'est PAS aléatoire
## mais au moins, on voit bien ce qu'il se passe.
def load_subSets(X, y, ratio_train=0.6, ratio_valid=0.2):
    ratio_test = 1 - ratio_train - ratio_valid #
    assert(ratio_test>0) ## on vérifie qu'il reste des exemples pour le test set
    Ntot   = X.shape[0]
    Ntrain = int(ratio_train*Ntot)
    Nvalid = int(ratio_valid*Ntot)
    Ntest  = Ntot - Ntrain - Nvalid
    X_train = X[0: Ntrain].copy()
    y_train = y[0: Ntrain].copy()
    X_valid = X[Ntrain:Ntrain+Nvalid].copy() # X[-Ntest:] est equivalent
    y_valid = y[Ntrain:Ntrain+Nvalid].copy() # X[-Ntest:] est equivalent
    X_test  = X[-Ntest:].copy()
    y_test  = y[-Ntest:].copy()
    assert(y_test.shape[0]>1) ## on vérifie qu'il reste des exemples pour le test set (on re-vérifie à cause de l'arrondi...)
    return X_train, y_train, X_valid, y_valid, X_test, y_test

## Partie 0 : choix du dataset

Pour vous guider, vous pouvez consulter les pages 19-20 du poly, qui sont un énoncé synthétique, mais qui donne la vue d'ensemble.

### Dataset 1: très bien pour commencer mais très petit, donc les expériences donneront des résultat plutôt peu intéressants dessus.


In [ ]:
## data set 1 ##
K=10
import sklearn.datasets
X,y = sklearn.datasets.load_digits(n_class=K, return_X_y=True)
DimRepre = 8
ratio_train = 0.6 # on peut reduire ce nombre lorsqu'on utilise le gros data set, mnist70.npz
ratio_valid = 0.3 # on peut reduire ce nombre lorsqu'on utilise le gros data set, mnist70.npz


### Dataset 2: mieux car plus grand, donc résultats plus intéressants. Attention, les expériences peuvent mettre du temps à tourner. 

Il peut être intéressant de commencer vos recherche sur une version plus petite du dataset (en gardant que 10% des data par exemple), et quand votre code marche bien et que vous avez resséré la plage de valeurs des hyper-paramètre qui est vraisemblablement la bonne, faire une recherche plus précise sur l'ensemble du dataset.

Astuce: en particulier, n'utilisez pas un grand nombre de composantes PCA sur ce dataset (en tout cas ne faites pas PCA(grand n_comp) + featureMap_polynomiale(degré2) + SVM là dessus.

À aller chercher sur : https://gitlab.inria.fr/flandes/data-for-teaching

Dans le shell linux, faites: 
`wget https://gitlab.inria.fr/flandes/data-for-teaching/-/raw/master/mnist70.npz?inline=false`



In [ ]:
# data set 2 ##
LoadObject = np.load("/home/flandes/data/mnist70.npz") ## adresse à adapter à votre cas bien sur !
DimRepre = 28
X = LoadObject['X']
y = LoadObject['y']
del LoadObject
ratio_train = 0.1  ## !! ici on ne garde que 10% pour train, pour que ça tourne vite, lorsqu'on développe le code !
ratio_valid = 0.1

### Dataset 3: même remarques que pour le 2

À aller chercher sur : https://gitlab.inria.fr/flandes/data-for-teaching



In [ ]:
# ## data set 3 ##
# LoadObject = np.load("fashion-mnist-reshaped.npz")
# DimRepre = 28
# X = LoadObject['train_images']
# y = LoadObject['train_labels']
# Xtest = LoadObject['test_images']
# ytest = LoadObject['test_labels']
# X     = np.array(X    , dtype=float) ## on change le type, car c'est unsigned-int (uint) au depart
# Xtest = np.array(Xtest, dtype=float)
# del LoadObject
# ratio_train = 0.1
# ratio_valid = 0.1

## Partie 1 --  PCA: prise en main
Ici, on souhaite faire une première PCA sur les données de MNIST (où n’importe quelles données visuelles).
1. Consultez la partie 1 du TP (fichier .ipynb) [vous y êtes !]
2. Calculer la transformée en PCA des données en conservant 95% de la variance des données. Doit-on
calculer la PCA seulement sur les données d’entraînement, ou bien sur l’ensemble (train+validation) ?
Pourquoi ? (Quel est le risque, dans le mauvais cas ?)
3. Extraire le nombre de composantes effectivement retenues par la décomposition.
4. [on vous donne le code] observer le tracé du ratio de variance expliquée en fonction du nombre de composantes. 
5. À partir de la version transformée des données, calculer aussi leur version décompressée (par transformée
inverse). Afficher quelques exemples de avant/après (compression+décompression). On peut jouer avec
le taux de variance expliquée et voir l’effet sur les images. (il est recommandé d’écrire une fonction).
6. (optionnel) Calculer l’erreur de reconstruction moyenne sur toutes les images (c’est plus facile que pour une seule
image). Le faire aussi pour l’ensemble de valdiation.
7. (optionnel) Calculer l’erreur de reconstruciton pour une seule image. (inclure ceci dans la fonction de comparaison
avant/après).

Remarque: la variance expliquée est le conjugué de l'erreur quadratique moyenne de reconstruction. 
En effet, la variance (expliquée) des données correspond à la dispersion (restante) entre les valeurs des pixels (moyennée sur l’ensemble du dataset).
L’erreur de reconstruction mesure le taux d'erreur (quadratique moyenne), entre l’image de départ et l ímage reconstruite, pixel par pixel (et moyennée ensuite sur toutes les images). 
Ce sont deux notions différentes mais qui sont en correspondance: une plus grande variance expliquée correspond toujours à une plus petite erreur de reconstruction.

In [ ]:
plt.figure()
n=41 # image numero 42
plt.imshow(X[n].reshape(DimRepre, DimRepre) , cm.gray)
plt.title("ceci est censé ressembler à un "+str(y[n])+ " avant la binarisation")


In [ ]:
X.max()

In [ ]:

# on met X (les intensités de niveaux de gris) entre 0 et 1
# (pour certains datasets, c'est nécessaire, pour d'autres c'est déjà fait)
X /= ?? # 

In [ ]:
ratio_train = ??
ratio ??
X_train, y_train, X_valid, y_valid, X_test, y_test = load_subSets(??)

### Ici, allez voir la doc de `sklearn.decomposition.PCA`

en particulier, il y est question d'un argument d'entrée `n_components` et d'un attribut  `explained_variance_ratio_`

In [ ]:
?sklearn.decomposition.PCA 
## c'est plus joli en ligne mais on a aussi la doc offline ! 
## pratique quand on ne se souvient plus du nom des arguments par exemple

In [ ]:
### 1er essai de PCA avec "ratio de variance expliquee" fixe a 0.95

# if premiere_partie_qq_essais_a_la_main:
varianceExplained = 0.95
preProc = sklearn.decomposition.PCA( ?? ) ## TODO: fixer les hyper-parametres de cette transformation
preProc.fit( ?? ) ## TODO: à éditer pour que ça marche

In [ ]:
## TODO: combien y a t il de composantes qui ont ete retenues ?


In [ ]:
## tracé de la variance expliquée au total pour tous les choix de n_component réalisables 
## (pour des variances expliquees entre 0 et 0.95)
CumulativeExplainedVariance = np.cumsum(preProc.explained_variance_ratio_)
plt.plot(CumulativeExplainedVariance, marker='+', label="ensemble de ?? TODO ??")
plt.xlabel("?? TODO ??")
plt.ylabel(" ?? TODO ??")
plt.ylim([0,1])
plt.legend()

# Explication complémentaire:

Petite explication sur l'utilisation du parametre `n_components` dans la classe PCA de sklearn :

Python est tres flexible, donc:
- si la valeur passée pour `n_components` est de type `int`, alors sklearn interprète cette valeur comme le nombre de composantes à garder dans la PCA (c'est le $D'$ de mon cours)
- si la valeur passée pour `n_components` est de type `float`  (et on espère, entre 0 et 1), sklearn l'interprète comme un ratio de variance expliquée désirée.
Dans ce 2ème cas, l'idée est la suivante: la variance des données est aussi égale à la somme des valeurs propres ($\lambda_1+\lambda_2+\ldots +\lambda_D$).
Si on garde par exemple que 2 composantes (les 2 premières), on aura une variance des données apres projection égale a $\lambda_1+\lambda_2$.
Le ratio de variance "expliquée"  (c.a.d. préservée par la PCA) sera de $$\frac{\lambda_1+\lambda_2}{\lambda_1+\lambda_2+\ldots +\lambda_D}$$
Ainsi,  si la valeur passée pour `n_components` est de type `float`, sklearn cherche des $\lambda$ jusqu'à ce que leur somme fasse 95% de la variance totale des données. Ainsi, il choisit le $D'$ à votre place... mais bon, vous avez choisi 95%, ce qui est aussi arbitraire (mais moins arbitraire que dire "gardons 10 composantes" ou bien "gardons 10% des composantes").

### Si vous êtes allés assez vite, vous pouvez faire tout ou partie de cette liste: 

(optionnel) 
- TODO: pour une image apres "compression" par PCA, calculer l'image décompressee correspondante.
- TODO: calculer ensuite l'erreur de reconstruction (erreur quadratique moyenne sur les pixels) pour une image *[éventuellement, faites ceci dans une fonction pour pouvoir rapidement comparer visuellement n'importe quel exemple d'entrainement avant et apres compression/decompression.]*
- TODO: calculer la version decompressée de tout le train-set.
- TODO : calculer l'erreur quadratique moyenne sur tout le train-set (indice: ca se fait en 1 seule ligne)
(vous pouvez traiter directement tout le train set, c'est en fait plus simple)

### Sinon, passez à la suite:

## Partie 2 -- Optimisation du nombre de composantes

Maintenant qu'on a compris un peu comment fonctionne la PCA, on va l'utiliser comme pré-processing:

1. Consultez la partie 2 du TP (fichier .ipynb) [vous y êtes !]
2. En faisant varier explicitement le nombre de composantes (nComp) à retenir dans la PCA, observez
comment les performances dépendent de ce nombre de composantes.
3. Sur le graphe, indiquer le point de meilleur score (le code vous est déjà donné, mais esayez de la faire par
vous même/de bien comprendre le code).

## Plus en détail:

- Commencez par utiliser la PCA comme pré processing (prenez un nombre de composantes  `n_components`  arbitraire, peu importe combien) c.a.d, transformez les données, puis, sur ces données "compressées" (après PCA), utilisez un classifieur:
    + consultez la doc de `sklearn.svm.SVC`
    + créer un SVM avec noyau (`kernel`) polynomial de degré 2
    + faites le fit de ce modèle, sur les données d'entrainement (transformées)
    + évaluez le modèle (calculez le score) sur les données d'entrainement (transformées)
    + évaluez le modèle (calculez le score) sur les données de validation (transformées aussi: essayez sans, vous verrez :P)
- une fois que ça marche pour un choix de `n_components` arbitraire, faites une boucle pour comparer plein de valeurs de `n_components`

### Conseil: 
commencez avec le dataset 1 !! Votre code tournera vite, c'est pratique pour développer :)
Et pour le dataset 2, n'allez pas jusqu'à D=784 composantes. Arretez vous à 64 par exemple, en allant de 4 en 4.

In [ ]:
ratio_train = ??
ratio_valid = ??
X_train, y_train, X_valid, y_valid, X_test, y_test = load_subSets(X, y, ratio_train, ratio_valid)

In [ ]:
preProc = sklearn.decomposition.PCA( ?? ) ## TODO
preProc.fit( ?? )  ## TODO
## TODO: transformer les donnees selon cette transformation apprise (comme à l'étape d'avant)

X_train_Transformed = ?? # TODO

In [ ]:
## TODO: faire l'apprentissage supervisé des classes, à l'aide d'un SVM polynomial de degre 2 (et avec C=1, coef0 = 1)
# clf = ## TODO
clf.fit(X_train_Transformed, y_train)  ## utilisez les données transformées 

In [ ]:
## TODO: calculer les scores a l'aide de clf.score() :
# trainscore = # TODO
# validscore = # TODO
# print("nombre Composants", nC , "   training score:",trainscore, ". valid score:", validscore)

#### Maintenant, mettons tout ça dans une boucle sur `n_components`

In [ ]:
linear_training_score = []
linear_valid_score = []
X_train, y_train, X_valid, y_valid, X_test, y_test = load_subSets(X,y)
## TODO: pour differentes valeurs de n_components, realiser une PCA
## puis une classification
nComp_range =
for nC in nComp_range:
    ## TODO: la meme chose qu'au dessus, 
    ## + enregistrer les resultats (train, valid scores)

In [ ]:
## plot des scores en fonction de l'hyper-param. nc
## c'est cadeau (rien à faire ici)
plt.figure()
plt.plot(nComp_range, linear_training_score, label= "train score")
plt.plot(nComp_range, linear_valid_score   , label= "valid score")
plt.xlabel("nombre comp")
plt.ylabel("scores")
plt.legend()
plt.ylim([0.5,1])

### Récupération de l'indice du meilleur choix de l'hyper-paramètre `n_components`

Ici on fait l'affichage du meilleur choix d'hyper-param (pour l'ensemble de validation évidemment).
Ici on vous montre un choix primaire, sans tenir compte de l'overfitting, etc: il peut et doit être affiné en observant les courbes (choix "à la main", à l'aide de tracés)

In [ ]:
bestIndex = np.argmax(linear_valid_score) ## on trouve l'indice tel que la performance est la meilleure
bestNC = nComp_range[bestIndex] ## on récupère la valeur de l'hyper-param qui correspond
plt.plot(bestNC, linear_valid_score[bestIndex], marker='X', color='green') ## on affiche le point

## Partie 2 bis: Cross-Validation

On a fait tout ceci pour un partage donné du train et validation set.

Pour être robuste vis-à-vis de l'aléatoire associé au tirage des données, il vaut mieux utiliser la cross-validation. Ceci est vrai en particulier pour les jeux de données avec peu d'exemple (N petit)

- faites un partage (tran+validation)/test des données, de façon déterministe (pas aléatoire), en vous inspirant de la fonction `load_subSets` donnée plus haut.
- faites vous meme une fonction de Cross-Validation:
    + faire un tableaux d'indices de 0 à N-1, mais aléatoire (mélangez les indices. Ça se dit *shuffle* en anglais) 
    + faites une méthode qui permet de piocher de façon déterminsite une tranche parmi 5 (par exemple), en choisissant la tranche
    + testez votre fonction (vérifier que quand on demande la tranche 0, puis la 1, ...  la 4, ça donne bien tout les exemples du datset, chacun étant vu une seule fois en tant que validation
- Utilisez votre méthode CrossValidation:
- pour une valeur de l'hyper-paramètre, faire 5 tirages train/validation, et mesurer les 5 scores (train+val, donc 10); enregistrer tout ces résultats
- faire à nouveau la boucle sur les valeurs de l'hyper-paramètre; enregistrer tout 

## Partie 3 -- Optimisation double: nComp et C
1. Consultez la partie 3 du TP (fichier .ipynb) [vous y êtes!]
2. Vous devez chercher à optimiser à la fois `n_components` et le paramètre `C` asocié à la réglarisation du SVM. Une
grosse part du travail à déjà été fait. Prenez garde à appliquer la transformation PCA au bon moment,
de la bonne façon.
3. Le tracé de la figure (score en fonction des deux hyper-paramètres) est déjà fourni. Lisez le attentivement
(vous pouvez aussi l’améliorer, par exemple en faisant un tracé "en 3D").
4. En cherchant dans la doc de np.argmax et de np.unravel, obtenez les coordonnés du point qui réalise le
maximum de score, et affichez le.
5. (optionnel) Refaire tout cela mais en utilisant par exemple le modèle des k plus proches voisins (k-NN, cf. la doc. de la classe `sklearn.neighbors.KNeighborsClassifier`, en faisant varier le nombre de voisins au lieu de C).

In [ ]:
nComp_range = np.arange(1,64,4)
hyperParam_range = [10**k  for k in range(-4,4,1)]
kernel = 'poly'
degree = 2

## si vous suivez cette convention d'enregistrement, le code (donné ci dessous) pour afficher le résultat
## fonctionnera directement. Si vous choisissez une autre convention, il faudra adapter le code..
array_training_score = np.zeros( (len(nComp_range), len(hyperParam_range)) )
array_validati_score = np.zeros( (len(nComp_range), len(hyperParam_range)) )

ratio_train = 0.4
ratio_valid = 0.4
X_train, y_train, X_valid, y_valid, X_test, y_test = load_subSets(X, y, ratio_train, ratio_valid)

In [ ]:
for i, nC in enumerate(nComp_range):
    X_train_Transformed = ?? # TODO
    ## TODO : completer ici
    for j, mu in enumerate(hyperParam_range):
        ## TODO: completer ici
        array_training_score[i,j] = trainscore
        array_validati_score[i,j] = validscore

#### On vous donne le code qui affiche la figure (si vous avez suivi ma façon d'enregistrer les résultats)

In [ ]:
### trace de la figure ###
plt.figure()
levels=np.array([0.5,0.6,0.7,0.8,0.9,0.95,1])
# levels=np.arange(0.8,1.0,0.01) ## pour un trace plus fin
plt.contourf(array_validati_score, levels=levels)
architecture = "SVM" ## on pourrait essayer d'autres classifieurs, comme k-NN par exemple.
if architecture=='SVM':
    nameofHyperParam = 'C (SVM)'
else:
    nameofHyperParam = 'k (k-NN)'
plt.xlabel("hyper-param "+nameofHyperParam)
xtick = np.arange(len(hyperParam_range))
plt.xticks(xtick, hyperParam_range)
plt.ylabel("num. of Comp (PCA)")
ytick = np.arange(len(nComp_range))
plt.yticks(ytick, nComp_range)
plt.colorbar()
# indice_max = ## TODO chercher par vous meme (ou stackoverflow) comment trouver le argmax d'un tableau a deux entrées
# plt.scatter(indice_max[1],indice_max[0] , marker='X', color='red')
plt.savefig("hyper-param-tuning_arci="+architecture+".png")



## Partie 4: avec plus de sklearn

Il était utile de faire tout ça à la main une fois, pour bien comprendre la validation, le pré-processing, mais :
- en fait, on peut automatiser tout ça, par exemple avec `sklearn.model_selection.cross_validate`  et `Pipeline`
- lisez la doc et faites tout ça en qq lignes !
- enjoy !